In [32]:
# stretching the cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# cuda settings
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.50)

In [33]:
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Defining global variables

In [51]:
image_size = slim.nets.vgg.vgg_16.default_image_size
num_classes = 2
num_channels = 3
train_data_path = "catvdog_train.tfrecords"
valid_data_path = "catvdog_valid.tfrecords"

## View all the layers

In [35]:
# vgg = slim.nets.vgg
# x = tf.placeholder(tf.float32,[None,224,224,3])
# with slim.arg_scope(vgg.vgg_arg_scope()):
#     _,_ = vgg.vgg_16(x,100)
# slim.get_variables_to_restore()

## Reading the tfrecords

In [40]:
def read_data(data_path,num_epochs,batch_size):
    feature = {'image':tf.FixedLenFeature([],tf.string), 'label':tf.FixedLenFeature([],tf.int64)}
    with tf.name_scope('filename_queue'):
        filename_queue = tf.train.string_input_producer([data_path],num_epochs=num_epochs)

    with tf.name_scope('tfreader'):
        reader = tf.TFRecordReader()
        _,serialzed_example = reader.read(filename_queue)

        features = tf.parse_single_example(serialzed_example,features=feature)

    with tf.name_scope('input_editor'):
        image = tf.decode_raw(features["image"],tf.float32)
        label = tf.cast(features["label"],tf.int32)

        image = tf.reshape(image,[image_size,image_size,num_channels])
        label = tf.one_hot(label,depth=num_classes)
    
    with tf.name_scope('shufflreader'):
        images,labels = tf.train.shuffle_batch([image,label],batch_size=batch_size,capacity=100,num_threads=4,min_after_dequeue=50)

    return (images,labels)

## Creating the default graph

In [45]:
tf.reset_default_graph()

print("Loading default graph.....")
graph = tf.Graph()
with graph.as_default():
    
    model_path = "./vgg_16.ckpt"
    assert(os.path.isfile(model_path))

    # Defining the placeholder variables
    with tf.name_scope('x_placeholders'):
        x = tf.placeholder(tf.float32,shape=(None,image_size,image_size,3),name='x_true')
    with tf.name_scope('y_placeholders'):
        y = tf.placeholder(tf.int32,shape=(None,num_classes),name='y_true')
    
    # loading the vgg graph
    vgg = slim.nets.vgg
    with slim.arg_scope(vgg.vgg_arg_scope(weight_decay=0.0001)):
        logits,end_points = vgg.vgg_16(x,num_classes=num_classes,is_training=True)
        
        
    variables_to_restore = slim.get_variables_to_restore(exclude=['vgg_16/fc8'])
    init_fn = slim.assign_from_checkpoint_fn(model_path,variables_to_restore)

    with tf.name_scope('last_layer_variables'):
        fc8_variables = slim.get_variables('vgg_16/fc8')
        fc8_init = tf.variables_initializer(fc8_variables)

    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y))

    with tf.name_scope('last_layer_optimizer'):
        fc8_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        fc8_train_op = fc8_optimizer.minimize(loss,var_list=[fc8_variables])

    with tf.name_scope('full_optimizer'):
        full_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
        full_train_op = full_optimizer.minimize(loss)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    with tf.name_scope('prediction'):
        y_pred = tf.nn.softmax(logits,name='y_pred')
        # actual output
        actual = tf.argmax(y,1)
        # predicted output
        prediction = tf.argmax(y_pred,1)
        
#     writer=tf.summary.FileWriter('./catvdoggraph/')
#     writer.add_graph(graph)
    # tf.get_default_graph().finalize()  
    
    print("\nFinalized the graph ... !")


Loading default graph.....

Finalized the graph ... !


# Training

In [48]:
# checking if the convnet graph is same as the default graph
assert logits.graph == graph


with tf.Session(graph=graph,config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    saver = tf.train.Saver()
    images,labels = read_data(train_data_path,num_epochs=100,batch_size=50)
    valid_images,valid_labels = read_data(valid_data_path,num_epochs=50,batch_size=100)
    
    with tf.name_scope('initialization'):
        init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())  
        sess.run(init_op)

        init_fn(sess)
        sess.run(fc8_init)
    
    with tf.name_scope('multithreading'):
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
    
    writer = tf.summary.FileWriter(logdir='./catvdoggraph/',graph=sess.graph)

    print("\n  Started Retraining ... !\n")

    # Retraining last layer...
    
    for epoch in range(500):
        img,lbl = sess.run([images,labels])
        img = img.astype(np.uint8)
        sess.run(fc8_train_op,feed_dict={x:img,y:lbl})
#         if(epoch%10 == 0):
#             print("Loss after {} epoch : {:.2f}".format(epoch,sess.run(loss,feed_dict={x:img,y:lbl})))
        if(epoch %100 == 0):
            val_img,val_lbl = sess.run([valid_images,valid_labels])
            val_img = val_img.astype(np.uint8)
            print("Validation Accuracy after retraining for {} epoch : {:.2f}".format(epoch+1,sess.run(accuracy,feed_dict={x:val_img,y:val_lbl})))


    print("\n  Started Fine Tuning .... !\n")
    # Fine tuning all layers
    epoch=0
    try:
        while not coord.should_stop():
            img,lbl = sess.run([images,labels])
            sess.run(full_train_op,feed_dict={x:img,y:lbl})
            if(epoch%100 == 0):
#                 print("Loss after {} epoch : {}".format(epoch,sess.run(loss,feed_dict={x:img,y:lbl})))
                val_img,val_lbl = sess.run([valid_images,valid_labels])
                print("Validation accuracy after fune tuning for {} epoch : {}".format(epoch+1,sess.run(accuracy,feed_dict={x:val_img,y:val_lbl})))  
            epoch+=1
    except tf.errors.OutOfRangeError as e:
        coord.request_stop(e)
        
    
    saver.save(sess,'saved_models/catvdog.ckpt')

    print("\n Finished Training .../")
    writer.close()
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Restoring parameters from ./vgg_16.ckpt

  Started Retraining ... !

Validation Accuracy after retraining for 1 epoch : 0.50
Validation Accuracy after retraining for 101 epoch : 0.36
Validation Accuracy after retraining for 201 epoch : 0.67
Validation Accuracy after retraining for 301 epoch : 0.77
Validation Accuracy after retraining for 401 epoch : 0.37

  Started Fine Tuning .... !

Validation accuracy after fune tuning for 1 epoch : 0.8600000143051147
Validation accuracy after fune tuning for 101 epoch : 0.8299999833106995
Validation accuracy after fune tuning for 201 epoch : 0.6700000166893005
Validation accuracy after fune tuning for 301 epoch : 0.8100000023841858
Validation accuracy after fune tuning for 401 epoch : 0.8600000143051147
Validation accuracy after fune tuning for 501 epoch : 0.7799999713897705
Validation accuracy after fune tuning for 601 epoch : 0.9100000262260437
Validation accuracy after fune tuning for 701 epoch : 0.8999999761581421
Validation acc

# Experiment

In [43]:
whos

Variable               Type                        Data/Info
------------------------------------------------------------
HTML                   type                        <class 'IPython.core.display.HTML'>
accuracy               Tensor                      Tensor("accuracy/Mean:0",<...> shape=(), dtype=float32)
actual                 Tensor                      Tensor("prediction/ArgMax<...> shape=(?,), dtype=int64)
coord                  Coordinator                 <tensorflow.python.traini<...>object at 0x7f35e40e4748>
correct_prediction     Tensor                      Tensor("accuracy/Equal:0"<...>, shape=(?,), dtype=bool)
display                function                    <function display at 0x7f369c891950>
end_points             OrderedDict                 OrderedDict([('vgg_16/con<...>=(?, 8) dtype=float32>)])
epoch                  int                         99
fc8_init               Operation                   name: "last_layer_variabl<...>g_16/fc8/biases/Assign"\n
fc8_opt

In [49]:
!tensorboard --logdir="./catvdoggraph/"

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-05-30 13:28:50.199752: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2018-05-30 13:28:51.288142: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1356] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.683
pciBusID: 0000:17:00.0
totalMemory: 10.91GiB freeMemory: 145.38MiB
2018-05-30 13:28:51.288197: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-05-30 13:28:51.808954: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with 